In [27]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Ładowanie zmiennych środowiskowych
load_dotenv()

db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_database = os.getenv("DB_DATABASE")

conn = (
    f"mssql+pyodbc://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(conn)
print("Connection successful!")


Connection successful!


In [78]:
# Testy Query - L
query = """
SELECT
    sm.*,
    CAST(s.IDStacji AS INT) AS IDStacji,
    s.Nazwa
FROM
    [SredMies2000to2018] sm
INNER JOIN
    [Stacje] s
ON
    s.IDStacji = sm.stacja
"""

df = pd.read_sql_query(query, engine)
df["Miesiac"]
miasto = pd.Series(['Warszawa'], name="Nazwa")
max_temp = df["SredTemp"].max()
miasto
max_temp
df.describe()
df
df.info()
df[df["SredTemp"] > 10]
dane_2_miast = df[(df["Nazwa"].eq("Warszawa")) | (df["Nazwa"].eq("Zakopane"))]
dane_2_miast

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Miesiac   977 non-null    int64  
 1   NazwaM    977 non-null    object 
 2   stacja    977 non-null    int64  
 3   SredTemp  977 non-null    float64
 4   IDStacji  977 non-null    int64  
 5   Nazwa     977 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 45.9+ KB


,Miesiac,NazwaM,stacja,SredTemp,IDStacji,Nazwa
152,4,kwiecień,12625,9.176944,12625,Zakopane
182,12,grudzień,12625,-0.915659,12625,Zakopane
202,9,wrzesień,12625,13.665863,12625,Zakopane
215,1,styczeń,12625,-2.618158,12625,Zakopane
232,10,październik,12376,9.997074,12376,Warszawa
239,6,czerwiec,12625,16.899417,12625,Zakopane
431,12,grudzień,12376,0.116008,12376,Warszawa
443,9,wrzesień,12376,16.750273,12376,Warszawa
450,1,styczeń,12376,-2.351453,12376,Warszawa
477,10,październik,12625,9.274003,12625,Zakopane
